### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [73]:
# # 한국어 전처리를 위한
!pip install konlpy
# !pip install Twitter
# !pip install tqdm
!pip install gensim
# !pip install --upgrade jupyter ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [74]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용

In [75]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [76]:
# 데이터를 데이터프레임으로 로드
musical_data = pd.read_csv('../mock_test_csv/mock.csv', encoding='utf-8')# 데이터 파일 경로
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,NaN,뮤지컬 <실연남녀> 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 ...,어리버리 조폭 형제가 경영하는 “아트 퓨쳐산장”. 이들은 6개월 전 “아트 퓨쳐산장...
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,90분,“No.1 창작뮤지컬의 신화”뮤지컬 ‘김종욱찾기(연출/김지연)’는 창작뮤지컬계 블루...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디“운명은 항상...
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,130분,"뮤지컬 <첫사랑>은 어느 바닷가를 배경으로, 이루어지지 못한 첫사랑에 대한 아련한 ...",시간을 조금만 되돌릴 수 있다면…어릴 적부터 한 동네에서 자란 해수와 선이는 오누이...
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬!2003년 초연 이래로 16번의 ...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다. 하지만 희...
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,160분 (인터미션 : 20분),3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...
...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/11/06 ~ 2008/02/03,충무아트센터 소극장 블루,"김우형,윤공주,문성혁,임은영,문지원",7세이상,110분,컨/페/션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑...,그대 가슴속에 두고 와야 할 게 있습니다…그건 바로 사/랑/ 입니다피아노 연주와 신...
96,5270,비보이 퍼포먼스 마리오네트,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/01/12 ~ 2007/03/11,충무아트센터 소극장 블루,NaN,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인!!!타 기존 작품들과의 차별화 성공!2006년 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...
97,5299,마리아 마리아,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/08/23 ~ 2007/08/26,충무아트센터 대극장,"강효성,김보강,이승철",만 8세이상,NaN,2007 뮤지컬 마리아마리아는..뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없...,"ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자, 제사장들은 이에 위..."
98,6509,어린이뮤지컬 장화신은 고양이,http://ticketimage.interpark.com/PlayDictionar...,어린이/가족,2007/07/28 ~ 2007/08/19,국립중앙박물관 극장 용,NaN,만 5세이상,60분,러시아 푸슈킨(Pushkin) 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여...,NaN


In [77]:
# 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
for col in musical_data.columns:
    musical_data[col] = musical_data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# 결과 출력
print(musical_data)

   musical_id            title  \
0        3885             실연남녀   
1        3635           김종욱 찾기   
2        4829              첫사랑   
3        3322              미라클   
4        4424              대장금   
..        ...              ...   
95       5075              컨페션   
96       5270   비보이 퍼포먼스 마리오네트   
97       5299          마리아 마리아   
98       6509  어린이뮤지컬 장화신은 고양이   
99       1162            빨간모자2   

                                           poster_url  genre  \
0   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
1   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
2   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
3   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
4   httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
..                                                ...    ...   
95  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
96  httpticketimageinterparkcomPlayDictionaryDATAP...     창작   
97  httpticketi

In [78]:
musical_data.to_csv('../cleaned_data_mock.csv', index=False, encoding='utf-8')

### 텍스트 전처리

In [79]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [80]:
#텍스트를 전처리 하는 함수
#BOOK_INTRCN_CN: 전처리할 도서 줄거리
def preprocessingText(synopsis):
    #전처리한 결과를 저장 할 리스트
    stems = []
    #text 를 단어로 나누고 품사를 붙여서 리턴
    #stem=True : 단어의 기본형으로 변환  ex) 같습니다,같았습니다 -> 같다.  
    tagged_review = twitter.pos(synopsis,stem=True)

    #tagged에서 하나를 꺼내서 word,pos에 대입
    #단어는 word, 품사는 pos에 대입
    for word,pos in tagged_review:
        #len(word) : 단어의 글자수가 2글자 이상이면
        #품사가 Noun(명사) 또는 Adjective(형용사) 일경우에만 word(단어) 를 stems에 추가
        if pos=="Noun" or pos=='Adjective':
            stems.append(word)

    #리스트인 stems를 문자열로 만들어서 ( "".join(stems) ) 리턴       
    return " ".join(stems)

In [81]:
preprocessingText(musical_data['synopsis'][0])

'조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마비 산장 주인 때문 난감하다 그 은행강도 사건 수배 중인 데 다가 자신 산장 주인 범인 오인 수도 있다 때문 우선 산장 주인 시체 그 대신 산장 운영 새롭다 삶 시작 맹세 그 후 개월 후 손님 교통사고 취소 난감하다 차 여자 손님 지아 남자 손님 연오 손님 좋아하다 그것 잠시 이 두 사람 자살 위해 이 산장 방문 것 알 난감하다 다애 실연 지아 배신 복수 위해 아내 죽음 연오 슬픔 위해 두 사람 추억 장소 아트 퓨쳐 산장 산장 자살 것 목 연오 지아 이 자살 장면 목격 운수 재수 결사 두 사람 만약 이 자살 시체 발견 경찰 수사 것 자신 전적 두렵다 것 자살 실패하다 지아 연오 또 다시 자살 위해 애 운수 재수 지아 연오 자살 위해 온갖 방법 원한 그렇다 가운데 지아 연오 사랑 때문 입장 것 알 그 아픔 이해 연오 역시 지아 아픔 마음 이해 그 연합전선 형성 자신 자살 운수 재수 형제 대항 온갖 죽 방법 원한 마침내 운수 재수 산장 밖 죽 기회 지아 연오 산장 밖 이 것 보고 발 동동 운수 재수 마침내 운수 특단 조치 생각 그 자살 위해 재수 그 비법 과연 이 그 밤 그 원하다 것 손 것 자 자 한바탕 소동 자살 방지 프로젝트 시작'

In [82]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체 및 전처리
musical_data['synopsis_clear'] = musical_data['synopsis'].fillna('').apply(preprocessingText)

In [83]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = musical_data['synopsis'].tolist()

In [84]:
# 데이터프레임의 'synopsis' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = musical_data['synopsis'].apply(lambda x: preprocessingText(str(x))).fillna('')

In [85]:
tokenized_data

0     조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...
1     첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...
2     시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...
3     인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...
4     어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...
                            ...                        
95    그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...
96    이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...
97    예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...
98                                                     
99    앤 할머니 시간 뒤 과거 뒤 기차 추억 역 향 추억 역 도착 앤 할머니 마침내 뒤 ...
Name: synopsis, Length: 100, dtype: object

In [86]:
#BOOK_INTRCN_CN_clear의 마지막에 " " 추가
musical_data["synopsis_clear"] = musical_data['synopsis_clear'].astype(str) + " "

In [87]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: synopsis_clear, Length: 100, dtype: object

In [88]:
#df["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" ")) : BOOK_INTRCN_CN_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
musical_data["tokenized_data"] = musical_data["synopsis_clear"].apply(lambda data : data.split(" "))

In [89]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,..."
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,..."
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버..."
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,..."
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일..."
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸..."
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,..."
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]"


In [90]:
musical_data["tokenized_data"]

0     [조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...
1     [첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...
2     [시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...
3     [인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...
4     [어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...
                            ...                        
95    [그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...
96    [이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...
97    [예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...
98                                                 [, ]
99    [앤, 할머니, 시간, 뒤, 과거, 뒤, 기차, 추억, 역, 향, 추억, 역, 도착...
Name: tokenized_data, Length: 100, dtype: object

### 유사도 측정
- 특정 줄거리와 다른 모든 줄거리 간의 유사도를 측정

In [91]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [92]:
# 학습용 Word2Vec 모델 훈련(벡터화)
model = Word2Vec(musical_data["tokenized_data"], 
                 vector_size=100, #WordVector의 열의 수
                 window=3, # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2, # 사용할 단어의 최소 빈도(2회 이하 단어 무시)
                 sg=1) # 0: CBOW, 1: Skip-gram

In [93]:
model.save("word2vec_model2.bin")

In [94]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'그': 0,
 '있다': 1,
 '자신': 2,
 '것': 3,
 '사랑': 4,
 '그녀': 5,
 '': 6,
 '수': 7,
 '위해': 8,
 '없다': 9,
 '사람': 10,
 '이': 11,
 '샌디': 12,
 '예수': 13,
 '말': 14,
 '대니': 15,
 '다시': 16,
 '이야기': 17,
 '집': 18,
 '새롭다': 19,
 '친구': 20,
 '마리아': 21,
 '존': 22,
 '고양이': 23,
 '마음': 24,
 '킴': 25,
 '모습': 26,
 '시작': 27,
 '남자': 28,
 '속': 29,
 '일': 30,
 '서로': 31,
 '렌': 32,
 '날': 33,
 '노래': 34,
 '크리스': 35,
 '그렇다': 36,
 '여자': 37,
 '엄마': 38,
 '같다': 39,
 '둘': 40,
 '춤': 41,
 '때': 42,
 '더': 43,
 '하나': 44,
 '곳': 45,
 '도시': 46,
 '에스메랄다': 47,
 '아이': 48,
 '막': 49,
 '한편': 50,
 '알': 51,
 '생각': 52,
 '삶': 53,
 '마을': 54,
 '안': 55,
 '꿈': 56,
 '온': 57,
 '전': 58,
 '다른': 59,
 '두': 60,
 '등': 61,
 '파티': 62,
 '사이': 63,
 '아버지': 64,
 '모든': 65,
 '토니': 66,
 '후': 67,
 '아름답다': 68,
 '무대': 69,
 '앞': 70,
 '살': 71,
 '순간': 72,
 '클럽': 73,
 '나': 74,
 '모두': 75,
 '자기': 76,
 '공연': 77,
 '소리': 78,
 '어머니': 79,
 '간다': 80,
 '대한': 81,
 '때문': 82,
 '계속': 83,
 '뮤지컬': 84,
 '음악': 85,
 '새': 86,
 '중': 87,
 '사실': 88,
 '시간': 89,
 '학교': 90,
 '생활': 91,
 '노미': 92,
 '리조': 

In [95]:
#믿음의 인덱스(빈도수 순위 0이 가장 빈도수 높은 단어)
model.wv.key_to_index["할머니"]

473

In [96]:
#믿음과 비슷한 벡터를 가진 단어 조회
model.wv.most_similar("할머니")

[('없다', 0.9918352365493774),
 ('가장', 0.9916790127754211),
 ('마을', 0.9912651777267456),
 ('파티', 0.9911972880363464),
 ('그', 0.9911845326423645),
 ('있다', 0.9911730289459229),
 ('온', 0.991141140460968),
 ('삶', 0.9911312460899353),
 ('앞', 0.991126298904419),
 ('사실', 0.9910821318626404)]

In [97]:
#믿음의 WordVector 조회
model.wv["할머니"]

array([-0.03631442,  0.05262725,  0.00072064,  0.01212332,  0.01400925,
       -0.09389757,  0.03671326,  0.10685872, -0.02267639, -0.03536092,
       -0.02745009, -0.07102394, -0.05111932,  0.00745809,  0.02267637,
       -0.04557564, -0.00302843, -0.0601797 , -0.01378565, -0.10119452,
        0.01836199,  0.01412282,  0.02699644, -0.00411274, -0.01835822,
       -0.02545348, -0.04039571, -0.07323517, -0.05134178,  0.01564172,
        0.09986716,  0.00337243,  0.00694477, -0.00412549,  0.01568238,
        0.06785407, -0.01249919, -0.03605388, -0.02948034, -0.08297787,
       -0.00308933, -0.04831329, -0.02811417,  0.01025605,  0.0392866 ,
       -0.00854659, -0.05592638, -0.00122472,  0.02561391,  0.07595355,
        0.04553248, -0.02664437, -0.01599741, -0.00829131, -0.03566596,
        0.02428843,  0.0301189 , -0.02078131, -0.04858286,  0.01349226,
        0.02581328,  0.02238709, -0.02435461,  0.00475752, -0.07445164,
        0.05434399,  0.00708402,  0.0534071 , -0.04566511,  0.06

In [98]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = model.wv.key_to_index.keys()
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '그녀', '', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '새롭다', '친구', '마리아', '존', '고양이', '마음', '킴', '모습', '시작', '남자', '속', '일', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '엄마', '같다', '둘', '춤', '때', '더', '하나', '곳', '도시', '에스메랄다', '아이', '막', '한편', '알', '생각', '삶', '마을', '안', '꿈', '온', '전', '다른', '두', '등', '파티', '사이', '아버지', '모든', '토니', '후', '아름답다', '무대', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '간다', '대한', '때문', '계속', '뮤지컬', '음악', '새', '중', '사실', '시간', '학교', '생활', '노미', '리조', '희망', '위', '아니다', '행동', '소개', '눈', '듀터', '듀티율', '자살', '인생', '못', '궁', '목사', '이제', '모어', '동안', '이름', '추억', '밤', '좋아하다', '번', '가지', '손님', '도착', '자리', '통해', '록시', '콰', '또', '작품', '댄스', '주인', '내', '대해', '점점', '과연', '엔지니어', '관계', '감정', '모도', '최고', '딸', '파의', '뒤', '거리', '역시', '세상', '많다', '장금', '버몬', '어떻다', '벽', '향', '현실', '일상', '젤리', '찰리', '페뷔스', '여인', '인하다', '메리', '돈', '채', '이용', '분위기', '캣츠비', '소냐', '마지막', '운명', '끝', '결혼', '사이공', '지선', '이상', '선영'

In [99]:
word2vec_words

dict_keys(['그', '있다', '자신', '것', '사랑', '그녀', '', '수', '위해', '없다', '사람', '이', '샌디', '예수', '말', '대니', '다시', '이야기', '집', '새롭다', '친구', '마리아', '존', '고양이', '마음', '킴', '모습', '시작', '남자', '속', '일', '서로', '렌', '날', '노래', '크리스', '그렇다', '여자', '엄마', '같다', '둘', '춤', '때', '더', '하나', '곳', '도시', '에스메랄다', '아이', '막', '한편', '알', '생각', '삶', '마을', '안', '꿈', '온', '전', '다른', '두', '등', '파티', '사이', '아버지', '모든', '토니', '후', '아름답다', '무대', '앞', '살', '순간', '클럽', '나', '모두', '자기', '공연', '소리', '어머니', '간다', '대한', '때문', '계속', '뮤지컬', '음악', '새', '중', '사실', '시간', '학교', '생활', '노미', '리조', '희망', '위', '아니다', '행동', '소개', '눈', '듀터', '듀티율', '자살', '인생', '못', '궁', '목사', '이제', '모어', '동안', '이름', '추억', '밤', '좋아하다', '번', '가지', '손님', '도착', '자리', '통해', '록시', '콰', '또', '작품', '댄스', '주인', '내', '대해', '점점', '과연', '엔지니어', '관계', '감정', '모도', '최고', '딸', '파의', '뒤', '거리', '역시', '세상', '많다', '장금', '버몬', '어떻다', '벽', '향', '현실', '일상', '젤리', '찰리', '페뷔스', '여인', '인하다', '메리', '돈', '채', '이용', '분위기', '캣츠비', '소냐', '마지막', '운명', '끝', '결혼', '사이공', '지선', '이상', '선영'

In [100]:
for index in range(2):
    # 도서 번호 조회
    NUM = musical_data.loc[index,"musical_id"]
    print("NUM=",NUM)
    # 도서 제목 조회
    TITLE = musical_data.loc[index,"title"]
    print("TITLE=",TITLE)
    #줄거리 조회
    LINE = musical_data.loc[index,"tokenized_data"]
    print("LINE=",LINE)
    doc2vec = None
    count = 0
    for word in LINE: #줄거리에서 단어 1개씩 리턴
        print("word=",word, end="\t")
        if word in word2vec_words: #단어가 WordVector에 포함된 단어 이면
            count += 1 #개수 1증가
            if doc2vec is None: #doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word] #단어의 WordVector 값 대입
            else: #첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word] #단어의 WordVector 값 추가

    if  doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
    print()    
    print("="*100)
    print("NUM=",NUM,"TITLE=",TITLE,":doc2vec=",doc2vec)
    print("="*100)

NUM= 3885
TITLE= 실연남녀
LINE= ['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월', '전', '아트', '퓨쳐', '산장', '사채', '빚', '갑자기', '심장마비', '산장', '주인', '때문', '난감하다', '그', '은행강도', '사건', '수배', '중인', '데', '다가', '자신', '산장', '주인', '범인', '오인', '수도', '있다', '때문', '우선', '산장', '주인', '시체', '그', '대신', '산장', '운영', '새롭다', '삶', '시작', '맹세', '그', '후', '개월', '후', '손님', '교통사고', '취소', '난감하다', '차', '여자', '손님', '지아', '남자', '손님', '연오', '손님', '좋아하다', '그것', '잠시', '이', '두', '사람', '자살', '위해', '이', '산장', '방문', '것', '알', '난감하다', '다애', '실연', '지아', '배신', '복수', '위해', '아내', '죽음', '연오', '슬픔', '위해', '두', '사람', '추억', '장소', '아트', '퓨쳐', '산장', '산장', '자살', '것', '목', '연오', '지아', '이', '자살', '장면', '목격', '운수', '재수', '결사', '두', '사람', '만약', '이', '자살', '시체', '발견', '경찰', '수사', '것', '자신', '전적', '두렵다', '것', '자살', '실패하다', '지아', '연오', '또', '다시', '자살', '위해', '애', '운수', '재수', '지아', '연오', '자살', '위해', '온갖', '방법', '원한', '그렇다', '가운데', '지아', '연오', '사랑', '때문', '입장', '것', '알', '그', '아픔', '이해', '연오', '역시', '지아', '아픔', '마음', '이해', '그', '연합전선', '형성', '자신', '자살', 

In [101]:
#행의 수를 row_num에 저장
row_num = len(musical_data)
row_num

100

In [102]:
# 1. doc2vec 값을 문자열로 변환
doc2vec_str = doc2vec.tostring()
doc2vec_str

/tmp/ipykernel_72326/669584216.py:2: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  doc2vec_str = doc2vec.tostring()


b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd4;(\x13\xf8<\xf3\x1e\x04\xbe\xd3j"=\x8fh\x0c>9\x954\xbd\xdb\xed\'\xbd\xa0\xc6\xee\xbc\xf7\t\xb0\xbd\x18\x8bn\xbd\xce\xf1O<y\xd5\x99<\xf6}g\xbd|\xea\xfb\xba\x12g\x96\xbdt\x87\xa9\xbck"\t\xbe\xc6+\x17=\x84\x8b\x96<\x8d=A=\xce\xd6\x0e\xbc\xd31\xf4\xbc\xbeH\xf4\xbc\xc2\x98H\xbdp6\xba\xbd\xcd\xbd\xa5\xbdQ\x03w<\x11o\x00>\xc7&\xc6\xbb\xd4;\x98<\xf8Nx\xbcX\x13;<\\\xee\xa7=\xc2{\xbf\xbcq\x0fc\xbdXt\x1f\xbdD\xe3\xfc\xbd\x88\xf7\x1a\xbc\x08\x15\x95\xbd\xbav\x1a\xbd\xcf\x9bl<x$w=\xec\x10\xd4\xbc|\xd8\x89\xbdC\xf8[;C\x1b\x00=\xa9\x96\xb6=fHj=D\xf32\xbd\xba\x8a\xc7\xbc\xe4\xa8\x93\xbcF(\x80\xbd#\x8f"=\x0510=\xbdO\x9c\xbc\x901\x88\xbd1h\xd3;\xd1\xc3/=\xfc\xdc\x8a<\xa2\xc9\xf7\xbc\x9f\x1f\x86<F\x8c\xe7\xbdX\xdd\x9c=\xfe\xa3\xe5;h\xca\xa6=2Hn\xbd\xc8=\xb5=Z\xd3\n\xbd\x95\x9a}==l\xe7=\xc8\x7fP\xbcc\x92\x8f=\x98\xe2\xaf<\r\xcbk\xbc\xd3;*\xbd\x90\xdd\x9c\xbde\xd1\xd4<(\xf3\x18\xbdT\x97\xc5\xbb8\x9fF\xbd\x98\x85\xaa=\xad\xeb\xd1\xb8\xf5\x06\xba:\xb6n\xb5<\xda\

In [103]:
# 2. 데이터프레임에 추가
musical_data['doc2vec_str'] = doc2vec_str

In [104]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_str
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...


#### date를 start_date와 end_date로 분리

In [105]:
# 날짜 형식에 맞게 잘라서 "YYYY-MM-DD" 또는 "YYYY-MM" 형식의 문자열로 만듭니다.
musical_data['start_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6] + '-' + musical_data['date'].str[6:8]
musical_data['end_date'] = musical_data['date'].str[:4] + '-' + musical_data['date'].str[4:6]

In [106]:
# datetime 객체로 변환
musical_data['start_date'] = pd.to_datetime(musical_data['start_date'], format="%Y-%m-%d")
musical_data['end_date'] = pd.to_datetime(musical_data['end_date'], format="%Y-%m")

In [107]:
musical_data

,musical_id,title,poster_url,genre,date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_str,start_date,end_date
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-10-13,2007-10-01
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2006-12-12,2006-12-01
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-03-27,2007-03-01
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-06-01,2007-06-01
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-05-26,2007-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-11-06,2007-11-01
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-01-12,2007-01-01
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-08-23,2007-08-01
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...,2007-07-28,2007-07-01


In [108]:
# 'date'와 'location' 사이로 'start_date'와 'end_date' 위치 변경
musical_data = musical_data[['musical_id', 'title', 'poster_url', 'genre', 'date', 'start_date', 'end_date', 'location',
                             'actors', 'age_rating', 'running_time', 'describe', 'synopsis', 'synopsis_clear',
                             'tokenized_data', 'doc2vec_str']]
musical_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,running_time,describe,synopsis,synopsis_clear,tokenized_data,doc2vec_str
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-13,2007-10-01,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,nan,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"[조폭, 형제, 경영, 아트, 퓨쳐, 산장, 이, 개월, 전, 아트, 퓨쳐, 산장,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-12,2006-12-01,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,90분,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"[첫사랑, 나선, 여자, 첫사랑, 남자, 티격태격, 로맨틱, 코미디, 운명, 항상,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-27,2007-03-01,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,130분,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"[시간, 조금, 수, 있다, 적, 동네, 해수, 오누이, 같다, 연인, 해수, 아버...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,90분,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"[인기, 그룹, 핫바, 멤버, 가수, 희다, 은, 날, 교통사고, 식물인간, 희다,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-26,2007-05-01,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,160분 인터미션 20분,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"[어머니, 유언, 위해, 궁, 생각, 시가, 장금, 것, 것, 좋아하다, 아이, 궁...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,컨페션,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071106 20080203,2007-11-06,2007-11-01,충무아트센터 소극장 블루,김우형윤공주문성혁임은영문지원,7세이상,110분,컨페션_그 두 번째 이야기충무아트홀은 오는 11월 6일부터 젊은이들의 꿈과 사랑 음...,그대 가슴속에 두고 와야 할 게 있습니다그건 바로 사랑 입니다피아노 연주와 신나는 ...,그대 가슴속 게 있다 바로 사랑 이다 피아노 연주 신나다 칵테일 쇼 있다 춘천 카페...,"[그대, 가슴속, 게, 있다, 바로, 사랑, 이다, 피아노, 연주, 신나다, 칵테일...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
96,5270,비보이 퍼포먼스 마리오네트,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070112 20070311,2007-01-12,2007-01-01,충무아트센터 소극장 블루,nan,만 6세이상,100분,비보이 퍼포먼스의 발전 가능성 확인타 기존 작품들과의 차별화 성공2006년 상반기 ...,이 공연은 총 6막으로 구성되어 인형사와 마리오네트 인형과의 사랑과 슬픔 등을 몸짓...,이 공연 총 막 구성 인형사 마리오네트 인형 사랑 슬픔 등 몸짓 표현 마리오네트 제...,"[이, 공연, 총, 막, 구성, 인형사, 마리오네트, 인형, 사랑, 슬픔, 등, 몸...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
97,5299,마리아 마리아,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070823 20070826,2007-08-23,2007-08-01,충무아트센터 대극장,강효성김보강이승철,만 8세이상,nan,2007 뮤지컬 마리아마리아는뮤지컬 마리아마리아의 작품 세계는 회화의 경계가 없다세...,ACT1예수와 그 제자들 주위에 따르는 군중들이 점점 늘어나자 제사장들은 이에 위협...,예수 그 제자 주위 군중 점점 제사장 이 위협 예수 제거 그 사주 바리새인 창녀 마...,"[예수, 그, 제자, 주위, 군중, 점점, 제사장, 이, 위협, 예수, 제거, 그,...",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...
98,6509,어린이뮤지컬 장화신은 고양이,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20070728 20070819,2007-07-28,2007-07-01,국립중앙박물관 극장 용,nan,만 5세이상,60분,러시아 푸슈킨Pushkin 극장 대표 레퍼토리어른과 어린이를 동시에 만족시킬 여름방...,nan,,"[, ]",b'\xed\xe72\xbd\x10\x92\xa3=8\xcf\xc2\xbbt{\xd...


In [109]:
# # musical_data.to_csv('../musical_data_vector.csv', index=False, encoding='utf-8') # 벡터변환 csv저장
musical_data.to_csv('../mock_test_csv/musical_data_vector_date.csv', index=False, encoding='utf-8')